In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np
from scipy.spatial import distance_matrix 
import os
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = "1"

In [4]:
[[1]] in [[1,2],[3,4]]

False

In [3]:
path = "./Training Problems"
os.listdir(path)

['problem1.txt',
 'problem10.txt',
 'problem11.txt',
 'problem12.txt',
 'problem13.txt',
 'problem14.txt',
 'problem15.txt',
 'problem16.txt',
 'problem17.txt',
 'problem18.txt',
 'problem19.txt',
 'problem2.txt',
 'problem20.txt',
 'problem3.txt',
 'problem4.txt',
 'problem5.txt',
 'problem6.txt',
 'problem7.txt',
 'problem8.txt',
 'problem9.txt']

In [12]:
paths_to_problems = []
for file in os.listdir(path):
    paths_to_problems.append(path+'/'+file)

In [13]:
# replace paths_to_problems[0] with the argument value

pickups = [[0,0]]
drops = [[0,0]]
f = open(paths_to_problems[0], "r")
f.readline()
for x in f:
    pickup,drop = (x.split()[1:])
    pickup = list(eval(pickup))
    drop = list(eval(drop))
    pickups.append(pickup)
    drops.append(drop)

In [6]:
pickups[1],drops[1]

([-9.100071078494038, -48.89301103772511],
 [-116.78442279683607, 76.80147820713637])

In [14]:
dist_matrix = distance_matrix(pickups[:3], drops[:3], p=2)

In [15]:
pickup_deliveries = [[i,i] for i in range(1,len(dist_matrix))]
pickup_deliveries

[[1, 1], [2, 2]]

In each pair of pickup_deliveries, the first entry is the pickup location index and the second is dropoff index

In [16]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = dist_matrix
    data["pickups_deliveries"] = pickup_deliveries
    data["num_vehicles"] = 2
    data["depot"] = 0
    return data

In [17]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += f" {manager.IndexToNode(index)} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f"{manager.IndexToNode(index)}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        print(plan_output)
        total_distance += route_distance
    print(f"Total Distance of all routes: {total_distance}m")

In [18]:
def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]),
        data["num_vehicles"],
        data["depot"]
    )
    
    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    #adding fixed cost of 500 for each driver
    routing.SetFixedCostOfAllVehicles(500)

    # Add Distance constraint.
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        720,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Define Transportation Requests.
    for request in data["pickups_deliveries"]:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index)
        )
        routing.solver().Add(
            distance_dimension.CumulVar(pickup_index)
            <= distance_dimension.CumulVar(delivery_index)
        )

    #custom objective function
    # routing.SetObjective(500*routing.VehicleVarCount() + routing.GetGlobalSpanCostVar())
    
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    # print(solution)
    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

In [19]:
if __name__ == "__main__":
    main()

: 